# Automating the Google Cricket Game

## By Team Pysquad :
### 1. Pranav Todkar
### 2. Sanskar Dethe

# Lets move to the code

Importing all required libraries...

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import ImageGrab as IG
import time
import pyautogui as pag

Using Pillow library to take Screenshots...

In [2]:
def take_screenshot(bbox = None):
    screenshot  = IG.grab(bbox)
    screenshot = np.array(screenshot)
    # Convert into BGR so that it can be used in OpenCV
    screenshot = cv2.cvtColor(screenshot, cv2.COLOR_RGB2BGR)
    return screenshot

# Google_Cricket class

<br>
Used to make Player and Ball object

In [3]:
class Google_cricket :
    # Attributes:
    ''' Attributes
    1. template_image - template image given as input
    2. height - (auto) height of template image
    3. width - (auto) width of template image
    4. location - location in the cropped (focus) section of screen
    '''
    # Constructor
    def __init__(self, path):
        # Attributes of player
        template_image  = cv2.imread(path) # Read player image
        self.template_image = template_image
        self.height = template_image.shape[0]
        self.width = template_image.shape[1]
        self.location = None
    
    def find_in(self, screen_img):
        # Optional CCOEFF / SSD

        result = cv2.matchTemplate(screen_img, self.template_image, cv2.TM_CCOEFF_NORMED)
        # result = cv2.matchTemplate(screen_img, self.template_image, cv2.TM_SQDIFF_NORMED)
        min_value, max_value, min_loc, max_loc = cv2.minMaxLoc(result)

        top_left = max_loc
        # top_left = min_loc

        bottom_right = (max_loc[0]+self.width, max_loc[1]+self.height)
        # bottom_right = (min_loc[0]+self.width, min_loc[1]+self.height)

        # Reason for using Norm...
        if max_value > 0.6: # Player location precision
        # if min_value < 0.4:
            self.location = (top_left, bottom_right)
            return True
        else:
            self.location = None
            return False

In [4]:
# Player object

#---------------------------------------------------------
Player = Google_cricket("stump.png") # Fill player.png

#---------------------------------------------------------
Ball = [Google_cricket("ball1.png"), Google_cricket("ball2.png"), Google_cricket("ball3.png")] # Fill ball.png

RUN THIS AND START THE GAME IN BROWSER

# Let the Game begin

In [21]:
# Parameters

distanceThreshold = 80


while True:
    screen  = take_screenshot()

    # Finding player

    
    if Player.find_in(screen):
        if Player.location != None : # Logic not understood
            # Find location of main_panal
            topleft_x = int(Player.location[0][0] - 0.17 * Player.width) # 10/58 * 58
            topleft_y = int(Player.location[0][1] - 2.11 * Player.height)
            bottomRight_x = int (Player.location[1][0] + 1.26 * Player.width)
            bottomRight_y = int(Player.location[1][1])
            TopLeft = (topleft_x, topleft_y)
            BottomRight = (bottomRight_x, bottomRight_y)
            break

# Press space to start
pag.press("space")

while True :

    # Cropping small section for observation
    main_panal = take_screenshot(bbox = (TopLeft[0], TopLeft[1], BottomRight[0], BottomRight[1]))

    # No Effect of Speed
    

    if Player.find_in(main_panal):
        if Player.location:
            # Yellow rectange [Aroung stump]
            cv2.rectangle(main_panal, Player.location[0], Player.location[1], (0, 255, 0), 7)
    
    # Detect ball
    for ball_i in Ball:
        if ball_i.find_in(main_panal):
            # Blue rectangle [Around ball]
            cv2.rectangle(main_panal, ball_i.location[0], ball_i.location[1], (255, 0, 0), 7)

            if Player.location:
                verticalDistance = Player.location[0][1] - ball_i.location[1][1]
                # Diff between Top left y coordinate

                if verticalDistance <= distanceThreshold:
                    pag.press("space")
                    break


    # show selected area
    cv2.imshow("Screen", main_panal)

    # Only Q as exit key
    if( cv2.waitKey(1) == ord('q')) :
        cv2.destroyAllWindows()
        break
